In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# do all necessary imports for this chapter
import matplotlib.pyplot as plt
import numpy as np
from ncon import ncon
from scipy.optimize import minimize
from scipy.sparse.linalg import eigs, LinearOperator, gmres
from scipy.linalg import svd, polar, null_space
from functools import partial
from time import time
from tutorialFunctions import createMPS, normalizeMPS, fixedPoints, rightOrthonormalize, mixedCanonical, expVal2Uniform, expVal2Mixed
from densityFunctions import traceDistance

In [3]:
def uniformToRho(A):
    l, r = fixedPoints(A)
    rho = ncon([l, A, A, A.conj(), A.conj(), r],
                 ((2, 1), (1, -3, 4), (4, -4, 6), (2, -1, 7), (7, -2, 8), (6, 8)))
    return rho

In [4]:
def gradCenterTerms(hTilde, A, l=None, r=None):
    """
    Calculate the value of the center terms.
    
        Parameters
        ----------
        hTilde : np.array (d, d, d, d)
            reduced Hamiltonian,
            ordered topLeft-topRight-bottomLeft-bottomRight.
        A : np.array (D, d, D)
            normalized MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array (D, D), optional
            left fixed point of transfermatrix,
            normalized.
        r : np.array (D, D), optional
            right fixed point of transfermatrix,
            normalized.
        
        Returns
        -------
        term1 : np.array (D, d, D)
            first term of gradient,
            ordered left-mid-right.
        term2 : np.array (D, d, D)
            second term of gradient,
            ordered left-mid-right.
    """
    
    # calculate fixed points if not supplied
    if l is None or r is None:
        l, r = fixedPoints(A)
        
    # calculate first contraction
    term1 = ncon((l, r, A, A, np.conj(A), hTilde), ([-1, 1], [5, 7], [1, 3, 2], [2, 4, 5], [-3, 6, 7], [3, 4, -2, 6]))
    
    # calculate second contraction
    term2 = ncon((l, r, A, A, np.conj(A), hTilde), ([6, 1], [5, -3], [1, 3, 2], [2, 4, 5], [6, 7, -1], [3, 4, 7, -2]))

    
    return term1, term2

In [5]:
def gradCenterTermsAA(A, l=None, r=None):
    if l is None or r is None:
        l, r = fixedPoints(A)
    
    # # Contraction to calculate TrAA
    # tensors = [l, A, A, A.conj(), A.conj(), r, l, A, A, A.conj(), A.conj(), r]
    # edges = [
    #     (1, 2), (2, 6, 10), (10, 12, 14), (1, 5, 7), (7, 11, 13), (14, 13),
    #     (4, 3), (3, 5, 8), (8, 11, 15), (4, 6, 9), (9, 12, 16), (15, 16)
    # ]
    
    tensors = [l, A, A, A.conj(), r, l, A, A, A.conj(), A.conj(), r]
    edges = [
        (-1, 2), (2, 6, 10), (10, 12, 14), (-3, 11, 13), (14, 13),
        (4, 3), (3, -2, 8), (8, 11, 15), (4, 6, 9), (9, 12, 16), (15, 16)
    ]
    term1 = ncon(tensors, edges)
    
    tensors = [l, A, A, A.conj(), r, l, A, A, A.conj(), A.conj(), r]
    edges = [
        (1, 2), (2, 6, 10), (10, 12, 14), (1, 5, -1), (14, -3),
        (4, 3), (3, 5, 8), (8, -2, 15), (4, 6, 9), (9, 12, 16), (15, 16)
    ]
    term2 = ncon(tensors, edges)
    
    tensors = [l, A, A, A.conj(), A.conj(), r, l, A, A, A.conj(), r]
    edges = [
        (1, 2), (2, -2, 10), (10, 12, 14), (1, 5, 7), (7, 11, 13), (14, 13),
        (-1, 3), (3, 5, 8), (8, 11, 15), (-3, 12, 16), (15, 16)
    ]
    term3 = ncon(tensors, edges)
    
    tensors = [l, A, A, A.conj(), A.conj(), r, l, A, A, A.conj(), r]
    edges = [
        (1, 2), (2, 6, 10), (10, -2, 14), (1, 5, 7), (7, 11, 13), (14, 13),
        (4, 3), (3, 5, 8), (8, 11, 15), (4, 6, -1), (15, -3)
    ]
    term4 = ncon(tensors, edges)
    
    return term1, term2, term3, term4

In [6]:
def reducedHamUniform(h, A, l=None, r=None):
    """
    Regularize Hamiltonian such that its expectation value is 0.
    
        Parameters
        ----------
        h : np.array (d, d, d, d)
            Hamiltonian that needs to be reduced,
            ordered topLeft-topRight-bottomLeft-bottomRight.
        A : np.array (D, d, D)
            normalized MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array (D, D), optional
            left fixed point of transfermatrix,
            normalized.
        r : np.array (D, D), optional
            right fixed point of transfermatrix,
            normalized.
            
        Returns
        -------
        hTilde : np.array (d, d, d, d)
            reduced Hamiltonian,
            ordered topLeft-topRight-bottomLeft-bottomRight.
    """
    
    d = A.shape[1]
    
    # calculate fixed points if not supplied
    if l is None or r is None:
        l, r = fixedPoints(A)
    
    # calculate expectation value
    e = np.real(expVal2Uniform(h, A, l, r))
    
    # substract from hamiltonian
    hTilde = h - e * ncon((np.eye(d), np.eye(d)), ([-1, -3], [-2, -4]))
    
    return hTilde

In [7]:
def EtildeRight(A, l, r, v):
    """
    Implement the action of (1 - Etilde) on a right vector v.
    
        Parameters
        ----------
        A : np.array (D, d, D)
            normalized MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array (D, D), optional
            left fixed point of transfermatrix,
            normalized.
        r : np.array (D, D), optional
            right fixed point of transfermatrix,
            normalized.
        v : np.array (D**2)
            right matrix of size (D, D) on which
            (1 - Etilde) acts,
            given as a vector of size (D**2,)
        
        Returns
        -------
        vNew : np.array (D**2)
            result of action of (1 - Etilde)
            on a right matrix,
            given as a vector of size (D**2,)
    """
    
    D = A.shape[0]
    
    # reshape to matrix
    v = v.reshape(D, D)
        
    # transfermatrix contribution
    transfer = ncon((A, np.conj(A), v), ([-1, 2, 1], [-2, 2, 3], [1, 3]))

    # fixed point contribution
    fixed = np.trace(l @ v) * r

    # sum these with the contribution of the identity
    vNew = v - transfer + fixed

    return vNew.reshape((D ** 2))

In [8]:
def RhUniform(hTilde, A, l=None, r=None):
    """
    Find the partial contraction for Rh.
    
        Parameters
        ----------
        hTilde : np.array (d, d, d, d)
            reduced Hamiltonian,
            ordered topLeft-topRight-bottomLeft-bottomRight,
            renormalized.
        A : np.array (D, d, D)
            normalized MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array (D, D), optional
            left fixed point of transfermatrix,
            normalized.
        r : np.array (D, D), optional
            right fixed point of transfermatrix,
            normalized.
        
        Returns
        -------
        Rh : np.array (D, D)
            result of contraction,
            ordered top-bottom.
    """
    
    D = A.shape[0]
    
    # if l, r not specified, find fixed points
    if l is None or r is None:
        l, r = fixedPoints(A)
    
    # construct b, which is the matrix to the right of (1 - E)^P in the figure above
    b = ncon((r, A, A, np.conj(A), np.conj(A), hTilde), ([4, 5], [-1, 2, 1], [1, 3, 4], [-2, 8, 7], [7, 6, 5], [2, 3, 8, 6]))
    
    # solve Ax = b for x
    A = LinearOperator((D ** 2, D ** 2), matvec=partial(EtildeRight, A, l, r))
    Rh = gmres(A, b.reshape(D ** 2))[0]
    
    return Rh.reshape((D, D))

In [9]:
def gradLeftTerms(hTilde, A, l=None, r=None):
    """
    Calculate the value of the left terms.
    
        Parameters
        ----------
        hTilde : np.array (d, d, d, d)
            reduced Hamiltonian,
            ordered topLeft-topRight-bottomLeft-bottomRight,
            renormalized.
        A : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array (D, D), optional
            left fixed point of transfermatrix,
            normalized.
        r : np.array (D, D), optional
            right fixed point of transfermatrix,
            normalized.
        
        Returns
        -------
        leftTerms : np.array (D, d, D)
            left terms of gradient,
            ordered left-mid-right.
    """
    
    # if l, r not specified, find fixed points
    if l is None or r is None:
        l, r = fixedPoints(A)
    
    # calculate partial contraction
    Rh = RhUniform(hTilde, A, l, r)
    
    # calculate full contraction
    leftTerms = ncon((Rh, A, l), ([1, -3], [2, -2, 1], [-1, 2]))
    
    return leftTerms

In [10]:
def EtildeLeft(A, l, r, v):
    """
    Implement the action of (1 - Etilde) on a left vector v.
    
        Parameters
        ----------
        A : np.array (D, d, D)
            normalized MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array (D, D), optional
            left fixed point of transfermatrix,
            normalized.
        r : np.array (D, D), optional
            right fixed point of transfermatrix,
            normalized.
        v : np.array (D**2)
            left matrix of size (D, D) on which
            (1 - Etilde) acts,
            given as a vector of size (D**2,)
        
        Returns
        -------
        vNew : np.array (D**2)
            result of action of (1 - Etilde)
            on a left matrix,
            given as a vector of size (D**2,)
    """
    
    D = A.shape[0]
    
    # reshape to matrix
    v = v.reshape(D, D)

    # transfer matrix contribution
    transfer = ncon((v, A, np.conj(A)), ([3, 1], [1, 2, -2], [3, 2, -1]))

    # fixed point contribution
    fixed = np.trace(v @ r) * l

    # sum these with the contribution of the identity
    vNew = v - transfer + fixed

    return vNew.reshape((D ** 2))

In [11]:
def LhUniform(hTilde, A, l=None, r=None):
    """
    Find the partial contraction for Lh.
    
        Parameters
        ----------
        hTilde : np.array (d, d, d, d)
            reduced Hamiltonian,
            ordered topLeft-topRight-bottomLeft-bottomRight,
            renormalized.
        A : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array (D, D), optional
            left fixed point of transfermatrix,
            normalized.
        r : np.array (D, D), optional
            right fixed point of transfermatrix,
            normalized.
        
        Returns
        -------
        Lh : np.array (D, D)
            result of contraction,
            ordered bottom-top.
    """
    
    D = A.shape[0]
    
    # if l, r not specified, find fixed points
    if l is None or r is None:
        l, r = fixedPoints(A)
    
    # construct b, which is the matrix to the right of (1 - E)^P in the figure above
    b = ncon((l, A, A, np.conj(A), np.conj(A), hTilde), ([5, 1], [1, 3, 2], [2, 4, -2], [5, 6, 7], [7, 8, -1], [3, 4, 6, 8]))    
    
    # solve Ax = b for x
    A = LinearOperator((D ** 2, D ** 2), matvec=partial(EtildeLeft, A, l, r)) 
    Lh = gmres(A, b.reshape(D ** 2))[0]
    
    return Lh.reshape((D, D))

In [12]:
def gradRightTerms(hTilde, A, l=None, r=None):
    """
    Calculate the value of the right terms.
    
        Parameters
        ----------
        hTilde : np.array (d, d, d, d)
            reduced Hamiltonian,
            ordered topLeft-topRight-bottomLeft-bottomRight,
            renormalized.
        A : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array (D, D), optional
            left fixed point of transfermatrix,
            normalized.
        r : np.array (D, D), optional
            right fixed point of transfermatrix,
            normalized.
        
        Returns
        -------
        rightTerms : np.array (D, d, D)
            right terms of gradient,
            ordered left-mid-right.
    """
    
    # if l, r not specified, find fixed points
    if l is None or r is None:
        l, r = fixedPoints(A)
    
    # calculate partial contraction
    Lh = LhUniform(hTilde, A, l, r)
    
    # calculate full contraction
    rightTerms = ncon((Lh, A, r), ([-1, 1], [1, -2, 2], [2, -3]))
    
    return rightTerms

In [15]:
def gradient(h, A, l=None, r=None):
    """
    Calculate the gradient of the expectation value of h @ MPS A.
    
        Parameters
        ----------
        h : np.array (d, d, d, d)
            Hamiltonian,
            ordered topLeft-topRight-bottomLeft-bottomRight,
            renormalized.
        A : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array (D, D), optional
            left fixed point of transfermatrix,
            normalized.
        r : np.array (D, D), optional
            right fixed point of transfermatrix,
            normalized.
        
        Returns
        -------
        grad : np.array (D, d, D)
            Gradient,
            ordered left-mid-right.
    """
    
    # if l, r not specified, find fixed points
    if l is None or r is None:
        l, r = fixedPoints(A)
        
    # renormalize Hamiltonian and A
    # hTilde = reducedHamUniform(h, A, l, r)
    # hTildeA = reducedHamUniform(rhoA, A, l, r)
    hTilde = h
    hA = uniformToRho(A)
        
    # find terms
    centerTerm1, centerTerm2 = gradCenterTerms(hTilde, A, l, r)
    leftTerms = gradLeftTerms(hTilde, A, l, r)
    rightTerms = gradRightTerms(hTilde, A, l, r)
    
    centerTermAA1, centerTermAA2, centerTermAA3, centerTermAA4 = gradCenterTermsAA(A, l, r)
    leftTermsA = 2*gradLeftTerms(hA, A, l, r)
    rightTermsA = 2*gradRightTerms(hA, A, l, r)
    
    # grad = (centerTerm1A + centerTerm2A + leftTermsA + rightTermsA)

    grad = centerTermAA1 + centerTermAA2 + centerTermAA3 + centerTermAA4 + leftTermsA + rightTermsA
    # grad -= 2 * (centerTerm1 + centerTerm2)
    grad -= 2 * (centerTerm1 + centerTerm2 + leftTerms + rightTerms)
    
#     AAterms = centerTermAA1 + centerTermAA2 + centerTermAA3 + centerTermAA4
#     ABterms = centerTerm1 + centerTerm2# + leftTerms + rightTerms
#     print('AA Terms')
#     print(AAterms)
#     print()
#     print('AB Terms')
#     print(ABterms)
#     print()
#     print('AA Terms - AB Terms')
#     print(AAterms - 2*ABterms)
    
    return grad

In [16]:
def optimiseDensityGradDescent(h, D, eps=1e-1, A0=None, tol=1e-4, maxIter=1e4, verbose=True):
    """
    Find the ground state using gradient descent.
    
        Parameters
        ----------
        h : np.array (d, d, d, d)
            Hamiltonian to minimize,
            ordered topLeft-topRight-bottomLeft-bottomRight.
        D : int
            Bond dimension
        eps : float
            Stepsize.
        A0 : np.array (D, d, D)
            normalized MPS tensor with 3 legs,
            ordered left-bottom-right,
            initial guess.
        tol : float
            Tolerance for convergence criterium.
        
        Returns
        -------
        E : float
            expectation value @ minimum
        A : np.array (D, d, D)
            ground state MPS,
            ordered left-mid-right.
    """
    
    d = h.shape[0]
    
    # if no initial value, choose random
    if A0 is None:
        A0 = createMPS(D, d)
        A0 = normalizeMPS(A0)
    
    # calculate gradient
    g = gradient(h, A0)
    
    A = A0
    
    i = 0
    
    while not(np.linalg.norm(g) < tol):
        # do a step
        A = A - eps * g
        A = normalizeMPS(A)
        i += 1
        
        if verbose and not(i % 50):
            #E = np.real(expVal2Uniform(h, A))
            rhoA = uniformToRho(A)
            E = traceDistance(h, rhoA)
            print('iteration:\t{:d}\tdist:\t{:.12f}\tgradient norm:\t{:.4e}'.format(i, E, np.linalg.norm(g)))
        
        # calculate new gradient
        g = gradient(h, A)
        
        if i > maxIter:
            print('Warning: gradient descent did not converge!')
            break
    
    # calculate ground state energy
    # E = np.real(expVal2Uniform(h, A))
    rhoA = uniformToRho(A)
    E = traceDistance(h, rhoA)   
    
    return E, A

In [17]:
# Prepare state to optimise
d0, D0 = 2, 2
A0 = createMPS(D0, d0)
A0 = normalizeMPS(A0)

h = uniformToRho(A0)

In [18]:
d, D = 2, 2
A = createMPS(D, d)
A = normalizeMPS(A)
#A = A0

print('Gradient descent optimization:\n')
t0 = time()
E1, A1 = optimiseDensityGradDescent(h, D, eps=1e-1, A0=A, tol=1e-5, maxIter=1e4)
print('Time until convergence:', time()-t0, 's')
print('Computed trace dist:', E1, '\n')

Gradient descent optimization:

iteration:	50	dist:	0.003447720699	gradient norm:	3.0961e-02
iteration:	100	dist:	0.000411001688	gradient norm:	1.2411e-02
iteration:	150	dist:	0.000122853808	gradient norm:	1.1957e-02
iteration:	200	dist:	0.000084466700	gradient norm:	1.0712e-02
iteration:	250	dist:	0.000069510532	gradient norm:	9.5633e-03
iteration:	300	dist:	0.000059514846	gradient norm:	8.6217e-03
iteration:	350	dist:	0.000051981365	gradient norm:	7.8426e-03
iteration:	400	dist:	0.000046080327	gradient norm:	7.1870e-03
iteration:	450	dist:	0.000041349196	gradient norm:	6.6274e-03
iteration:	500	dist:	0.000037486835	gradient norm:	6.1438e-03
iteration:	550	dist:	0.000034285685	gradient norm:	5.7216e-03
iteration:	600	dist:	0.000031597860	gradient norm:	5.3497e-03
iteration:	650	dist:	0.000029315330	gradient norm:	5.0196e-03
iteration:	700	dist:	0.000027357533	gradient norm:	4.7246e-03
iteration:	750	dist:	0.000025663311	gradient norm:	4.4593e-03
iteration:	800	dist:	0.000024185499	gra

iteration:	6550	dist:	0.000006837361	gradient norm:	2.1555e-04
iteration:	6600	dist:	0.000006823287	gradient norm:	2.1082e-04
iteration:	6650	dist:	0.000006809438	gradient norm:	2.0618e-04
iteration:	6700	dist:	0.000006795807	gradient norm:	2.0161e-04
iteration:	6750	dist:	0.000006782391	gradient norm:	1.9711e-04
iteration:	6800	dist:	0.000006769183	gradient norm:	1.9269e-04
iteration:	6850	dist:	0.000006756179	gradient norm:	1.8835e-04
iteration:	6900	dist:	0.000006743374	gradient norm:	1.8408e-04
iteration:	6950	dist:	0.000006730764	gradient norm:	1.7988e-04
iteration:	7000	dist:	0.000006718372	gradient norm:	1.7562e-04
iteration:	7050	dist:	0.000006706120	gradient norm:	1.7168e-04
iteration:	7100	dist:	0.000006694066	gradient norm:	1.6769e-04
iteration:	7150	dist:	0.000006682190	gradient norm:	1.6376e-04
iteration:	7200	dist:	0.000006670489	gradient norm:	1.5989e-04
iteration:	7250	dist:	0.000006658957	gradient norm:	1.5609e-04
iteration:	7300	dist:	0.000006647591	gradient norm:	1.5

---
## Testing $Tr(\rho_A \rho_B)$

Want to make sure that the construction using tensor networks is the same as the construction using standard matrices. 

In [ ]:
d, D = 2, 4
A = createMPS(D, d)
A = normalizeMPS(A)

B = createMPS(D, d)
B = normalizeMPS(B)

rhoA = uniformToRho(A).reshape(d**2, d**2)
rhoB = uniformToRho(B).reshape(d**2, d**2)

TrAB = np.trace(rhoA @ rhoB)

In [ ]:
TrAB

In [ ]:
def tensorTrAB(A, B):
    
    lA, rA = fixedPoints(A)
    lB, rB = fixedPoints(B)
    tensors = [lB, B, B, B.conj(), B.conj(), rB, lA, A, A, A.conj(), A.conj(), rA]
    edges = [
        (1, 2), (2, 6, 10), (10, 12, 14), (1, 5, 7), (7, 11, 13), (14, 13),
        (4, 3), (3, 5, 8), (8, 11, 15), (4, 6, 9), (9, 12, 16), (15, 16)
    ]
    
    return ncon(tensors, edges)

In [ ]:
TrAB_tensor = tensorTrAB(A, B)

In [ ]:
TrAB_tensor

In [ ]:
print(np.allclose(TrAB, TrAB_tensor))

In [ ]:
traceDistExact = traceDistance(rhoA.reshape(d, d, d, d), rhoB.reshape(d, d, d, d))
traceDistExact

In [ ]:
traceDist_Tensor = tensorTrAB(A, A) + tensorTrAB(B, B) - 2*tensorTrAB(A, B)
traceDist_Tensor

In [ ]:
np.allclose(traceDistExact, traceDist_Tensor)

## Verify gradient is `0` when `A == B`



In [19]:
d, D = 2, 4
A = createMPS(D, d)
A = normalizeMPS(A)

rhoA = uniformToRho(A)

In [20]:
gradient(rhoA, A)

array([[[-1.11022302e-16-2.22044605e-16j,
          0.00000000e+00+0.00000000e+00j,
         -2.22044605e-16-2.22044605e-16j,
         -1.11022302e-16-2.22044605e-16j],
        [ 0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j,
         -2.22044605e-16-4.44089210e-16j,
          0.00000000e+00+0.00000000e+00j]],

       [[-1.11022302e-16-2.22044605e-16j,
         -2.22044605e-16+0.00000000e+00j,
          0.00000000e+00-2.22044605e-16j,
         -1.11022302e-16-2.22044605e-16j],
        [-2.22044605e-16+0.00000000e+00j,
          2.22044605e-16-1.11022302e-16j,
          0.00000000e+00-2.22044605e-16j,
          0.00000000e+00+0.00000000e+00j]],

       [[ 0.00000000e+00-1.11022302e-16j,
         -1.11022302e-16-2.22044605e-16j,
          0.00000000e+00-2.22044605e-16j,
         -1.11022302e-16+2.22044605e-16j],
        [ 0.00000000e+00-1.11022302e-16j,
         -2.22044605e-16+0.00000000e+00j,
          0.00000000e+00-2.22044605e-16j,
          0.00000000e+00-

In [21]:
np.linalg.norm(_)

1.1989637652951122e-15

## Verify left and right gradient terms for $Tr \rho_A \rho_A$

In [22]:
def EtildeRight(A, l, r, v):
    """
    Implement the action of (1 - Etilde) on a right vector v.
    
        Parameters
        ----------
        A : np.array (D, d, D)
            normalized MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array (D, D), optional
            left fixed point of transfermatrix,
            normalized.
        r : np.array (D, D), optional
            right fixed point of transfermatrix,
            normalized.
        v : np.array (D**2)
            right matrix of size (D, D) on which
            (1 - Etilde) acts,
            given as a vector of size (D**2,)
        
        Returns
        -------
        vNew : np.array (D**2)
            result of action of (1 - Etilde)
            on a right matrix,
            given as a vector of size (D**2,)
    """
    
    D = A.shape[0]
    
    # reshape to matrix
    v = v.reshape(D, D)
        
    # transfermatrix contribution
    transfer = ncon((A, np.conj(A), v), ([-1, 2, 1], [-2, 2, 3], [1, 3]))

    # fixed point contribution
    fixed = np.trace(l @ v) * r

    # sum these with the contribution of the identity
    vNew = v - transfer + fixed

    return vNew.reshape((D ** 2))

In [73]:
def EtildeRightInner(A, l, r, v):
    """
    Implement the action of (1 - Etilde) on a right vector v.
    
        Parameters
        ----------
        A : np.array (D, d, D)
            normalized MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array (D, D), optional
            left fixed point of transfermatrix,
            normalized.
        r : np.array (D, D), optional
            right fixed point of transfermatrix,
            normalized.
        v : np.array (D**2)
            right matrix of size (D, D) on which
            (1 - Etilde) acts,
            given as a vector of size (D**2,)
        
        Returns
        -------
        vNew : np.array (D**2)
            result of action of (1 - Etilde)
            on a right matrix,
            given as a vector of size (D**2,)
    """
    
    D = A.shape[0]
    
    # reshape to matrix
    v = v.reshape(D, D)
        
    # transfermatrix contribution
    transfer = ncon((np.conj(A), A, v), ([-1, 2, 1], [-2, 2, 3], [1, 3]))

    # fixed point contribution
    fixed = np.trace(l.T @ v) * r.T

    # sum these with the contribution of the identity
    vNew = v - transfer + fixed

    return vNew.reshape((D ** 2))

In [74]:
def RhAUniform(A, l=None, r=None):
    """
    Find the partial contraction for Rh.
    
        Parameters
        ----------
        hTilde : np.array (d, d, d, d)
            reduced Hamiltonian,
            ordered topLeft-topRight-bottomLeft-bottomRight,
            renormalized.
        A : np.array (D, d, D)
            normalized MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array (D, D), optional
            left fixed point of transfermatrix,
            normalized.
        r : np.array (D, D), optional
            right fixed point of transfermatrix,
            normalized.
        
        Returns
        -------
        Rh : np.array (D, D)
            result of contraction,
            ordered top-bottom.
    """
    
    D = A.shape[0]
    
    # if l, r not specified, find fixed points
    if l is None or r is None:
        l, r = fixedPoints(A)
    
    # construct b, which is the matrix to the right of (1 - E)^P in the figure above
    #b = ncon((r, A, A, np.conj(A), np.conj(A), hTilde), ([4, 5], [-1, 2, 1], [1, 3, 4], [-2, 8, 7], [7, 6, 5], [2, 3, 8, 6]))
    b = ncon((l, A, A, np.conj(A), np.conj(A), np.conj(A), np.conj(A), A, A, r, r), 
            ([1, 2], [-1, 3, 5], [5, 9, 11], [-2, 4, 8], [8, 10, 14], 
             [1, 3, 6], [6, 9, 12], [2, 4, 7], [7, 10, 13], [13, 12], [11, 14]))
    
    # solve Ax = b for x
    A = LinearOperator((D ** 2, D ** 2), matvec=partial(EtildeRight, A, l, r))
    Rh = gmres(A, b.reshape(D ** 2))[0]
    
    return Rh.reshape((D, D))

In [75]:
def RhAUniformInner(A, l=None, r=None):
    """
    Find the partial contraction for Rh.
    
        Parameters
        ----------
        hTilde : np.array (d, d, d, d)
            reduced Hamiltonian,
            ordered topLeft-topRight-bottomLeft-bottomRight,
            renormalized.
        A : np.array (D, d, D)
            normalized MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array (D, D), optional
            left fixed point of transfermatrix,
            normalized.
        r : np.array (D, D), optional
            right fixed point of transfermatrix,
            normalized.
        
        Returns
        -------
        Rh : np.array (D, D)
            result of contraction,
            ordered top-bottom.
    """
    
    D = A.shape[0]
    
    # if l, r not specified, find fixed points
    if l is None or r is None:
        l, r = fixedPoints(A)
    
    # construct b, which is the matrix to the right of (1 - E)^P in the figure above
    #b = ncon((r, A, A, np.conj(A), np.conj(A), hTilde), ([4, 5], [-1, 2, 1], [1, 3, 4], [-2, 8, 7], [7, 6, 5], [2, 3, 8, 6]))
    b = ncon((l, np.conj(A), np.conj(A), A, A, A, A, np.conj(A), np.conj(A), r, r), 
            ([2, 1], [-1, 3, 5], [5, 9, 11], [-2, 4, 8], [8, 10, 14], 
             [1, 3, 6], [6, 9, 12], [2, 4, 7], [7, 10, 13], [12, 13], [14, 11]))
    
    # solve Ax = b for x
    A = LinearOperator((D ** 2, D ** 2), matvec=partial(EtildeRightInner, A, l, r))
    Rh = gmres(A, b.reshape(D ** 2))[0]
    
    return Rh.reshape((D, D))

In [76]:
d, D = 2, 4
A = createMPS(D, d)
A = normalizeMPS(A)

rhoA = uniformToRho(A)
l, r = fixedPoints(A)

In [77]:
# Verify RhAUniform reproduces previous Rh
Rh = RhUniform(rhoA, A)
RhA = RhAUniform(A)

In [78]:
np.allclose(Rh, RhA)

True

In [79]:
bInner = ncon((l, np.conj(A), np.conj(A), A, A, A, A, np.conj(A), np.conj(A), r, r), 
            ([2, 1], [-1, 3, 5], [5, 9, 11], [-2, 4, 8], [8, 10, 14], 
             [1, 3, 6], [6, 9, 12], [2, 4, 7], [7, 10, 13], [12, 13], [14, 11]))

In [80]:
bA = ncon((l, A, A, np.conj(A), np.conj(A), np.conj(A), np.conj(A), A, A, r, r), 
        ([1, 2], [-1, 3, 5], [5, 9, 11], [-2, 4, 8], [8, 10, 14], 
         [1, 3, 6], [6, 9, 12], [2, 4, 7], [7, 10, 13], [13, 12], [11, 14]))

In [81]:
RhARHS = EtildeRight(A, l, r, RhA).reshape(4, 4)

In [82]:
np.allclose(RhARHS, bA)

True

In [83]:
RhInnerRHS = EtildeRightInner(A, l, r, RhA.T).reshape(4, 4)

In [84]:
np.allclose(RhInnerRHS, bInner)

True

In [85]:
# Calculate RhAUniformInner and see if it is RhAUniform.T
RhAInner = RhAUniformInner(A)

In [86]:
np.allclose(RhA, RhAInner.T)

True

In [87]:
def gradLeftTerms(hTilde, A, l=None, r=None):
    """
    Calculate the value of the left terms.
    
        Parameters
        ----------
        hTilde : np.array (d, d, d, d)
            reduced Hamiltonian,
            ordered topLeft-topRight-bottomLeft-bottomRight,
            renormalized.
        A : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array (D, D), optional
            left fixed point of transfermatrix,
            normalized.
        r : np.array (D, D), optional
            right fixed point of transfermatrix,
            normalized.
        
        Returns
        -------
        leftTerms : np.array (D, d, D)
            left terms of gradient,
            ordered left-mid-right.
    """
    
    # if l, r not specified, find fixed points
    if l is None or r is None:
        l, r = fixedPoints(A)
    
    # calculate partial contraction
    Rh = RhUniform(hTilde, A, l, r)
    
    # calculate full contraction
    leftTerms = ncon((Rh, A, l), ([1, -3], [2, -2, 1], [-1, 2]))
    
    return leftTerms

In [88]:
def gradLeftTermsA(A, l=None, r=None):
    """
    Calculate the value of the left terms.
    
        Parameters
        ----------
        A : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array (D, D), optional
            left fixed point of transfermatrix,
            normalized.
        r : np.array (D, D), optional
            right fixed point of transfermatrix,
            normalized.
        
        Returns
        -------
        leftTerms : np.array (D, d, D)
            left terms of gradient,
            ordered left-mid-right.
    """
    
    # if l, r not specified, find fixed points
    if l is None or r is None:
        l, r = fixedPoints(A)
    
    # calculate partial contraction
    Rh = RhAUniform(A, l, r)
    
    # calculate full contraction
    leftTerms = ncon((Rh, A, l), ([1, -3], [2, -2, 1], [-1, 2]))
    
    return leftTerms

In [89]:
gradL = gradLeftTerms(rhoA, A)
gradAL = gradLeftTermsA(A)

In [90]:
np.allclose(gradL, gradAL)

True

In [91]:
def gradLeftTermsAInner(A, l=None, r=None):
    """
    Calculate the value of the left terms.
    
        Parameters
        ----------
        A : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array (D, D), optional
            left fixed point of transfermatrix,
            normalized.
        r : np.array (D, D), optional
            right fixed point of transfermatrix,
            normalized.
        
        Returns
        -------
        leftTerms : np.array (D, d, D)
            left terms of gradient,
            ordered left-mid-right.
    """
    
    # if l, r not specified, find fixed points
    if l is None or r is None:
        l, r = fixedPoints(A)
    
    # calculate partial contraction
    Rh = RhAUniformInner(A, l, r)
    
    # calculate full contraction
    leftTerms = ncon((Rh, A, l), ([-3, 1], [2, -2, 1], [-1, 2]))
    
    return leftTerms

In [92]:
gradALInner = gradLeftTermsAInner(A)

In [93]:
np.allclose(gradALInner, gradAL)

True